In [2]:
#packages
library(readr)
library(tidyverse)
library(skimr)
library(glmnet)

In [3]:
#cleaning data
ts_raw <- read_csv("taylor_swift_spotify.csv")

ts <- ts_raw |>
  distinct(id, .keep_all = TRUE) |>      
  select(-`Unnamed: 0`) |>               
  rename(
    track_name = name,
    album_name = album
  ) |>
  mutate(
    release_date = as.Date(release_date),

    bw_album = case_when(
      album_name %in% c(
        "folklore",
        "folklore (deluxe version)",
        "folklore: the long pond studio sessions (from the Disney+ special) [deluxe edition]",
        "reputation",
        "reputation Stadium Tour Surprise Song Playlist",
        "THE TORTURED POETS DEPARTMENT",
        "THE TORTURED POETS DEPARTMENT: THE ANTHOLOGY"
      ) ~ "black_white",
      TRUE ~ "color"
    ),
    bw_album = factor(bw_album, levels = c("color", "black_white"))
  )

# audio feature columns used throughout
audio_vars <- c(
  "acousticness", "danceability", "energy", "instrumentalness",
  "liveness", "loudness", "speechiness", "tempo", "valence"
)

# drop rows with missing audio features or popularity (if any)
ts <- ts |>
  drop_na(all_of(c(audio_vars, "popularity")))

# popularity quartiles and groups
p25 <- quantile(ts$popularity, 0.25, na.rm = TRUE)
p75 <- quantile(ts$popularity, 0.75, na.rm = TRUE)

ts <- ts |>
  mutate(
    pop_group = case_when(
      popularity <= p25 ~ "low",
      popularity >= p75 ~ "high",
      TRUE ~ "medium"
    ),
    pop_group = factor(pop_group, levels = c("low", "medium", "high"))
  )

# quick check
glimpse(ts)


New names:
• `` -> `...1`
Rows: 582 Columns: 18
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (4): name, album, id, uri
dbl  (13): ...1, track_number, acousticness, danceability, energy, instrumen...
date  (1): release_date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


ERROR: [1m[33mError[39m in `select()`:[22m
[33m![39m Can't subset columns that don't exist.
[31m✖[39m Column `Unnamed: 0` doesn't exist.
